# IIC-2433 Minería de Datos UC

- Versiones de librerías, python 3.8.10

- numpy 1.20.3
- sklearn 1.0.2
- matplotlib 3.5.2
- scipy 1.8.0

In [1]:
from sklearn import datasets
digits = datasets.load_digits()
X = digits.data[:50,]
y = digits.target[:50,]


## Actividad en clase

Clusterice el dataset usando **HAC**. Para esto haga lo siguiente:

- Escale los datos usando MinMaxScaler y reduzca la dimensionalidad del sample de manera que retenga el 90% de la varianza.
- Corra HAC con {single link|average link|complete link|ward} para valores de k en el rango {8, 9, 10, 11, 12}. 
- Determine el mejor valor de k según el Silhouette score promedio. 
- Determine cuáles son los dos mejores modelos para el rango de k evaluado. 
- Cuanto termine, me avisa para entregarle una **L (logrado)**.
- Recuerde que las L otorgan un bono en la nota final de la asignatura.
- Pueden trabajar de a dos.

***Tiene hasta el final de la clase.***


# Solución

In [2]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

In [3]:
from sklearn.decomposition import PCA

def min_var(v,exp_var):
    acc = 0
    i=0
    while acc < v:
        acc += exp_var[i]
        i += 1
    return i

In [4]:
pca = PCA(n_components=50)
pca.fit(X)
min_var(0.9,pca.explained_variance_ratio_)

16

In [5]:
pca = PCA(n_components=16)
X= pca.fit_transform(X)

In [6]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

range_n_clusters = [8, 9, 10, 11, 12]

for n_clusters in range_n_clusters:
    
    hac1 = AgglomerativeClustering(linkage='single', n_clusters=n_clusters)
    hac1 = hac1.fit(X)

    hac2 = AgglomerativeClustering(linkage='average', n_clusters=n_clusters)
    hac2 = hac2.fit(X)

    hac3 = AgglomerativeClustering(linkage='complete', n_clusters=n_clusters)
    hac3 = hac3.fit(X)

    hac4 = AgglomerativeClustering(linkage='ward', n_clusters=n_clusters)
    hac4 = hac4.fit(X)

    
    cluster_labels_1 = hac1.fit_predict(X)
    cluster_labels_2 = hac2.fit_predict(X)
    cluster_labels_3 = hac3.fit_predict(X)
    cluster_labels_4 = hac4.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg_1 = silhouette_score(X, cluster_labels_1)
    silhouette_avg_2 = silhouette_score(X, cluster_labels_2)
    silhouette_avg_3 = silhouette_score(X, cluster_labels_3)
    silhouette_avg_4 = silhouette_score(X, cluster_labels_4)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score using single link is :",
        silhouette_avg_1,
    )
    
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score using average link is :",
        silhouette_avg_2,
    )
    
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score using complete link is :",
        silhouette_avg_3,
    )
    
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score using ward is :",
        silhouette_avg_4,
    )


For n_clusters = 8 The average silhouette_score using single link is : 0.21318737806351148
For n_clusters = 8 The average silhouette_score using average link is : 0.2922164553707203
For n_clusters = 8 The average silhouette_score using complete link is : 0.2725985259022848
For n_clusters = 8 The average silhouette_score using ward is : 0.30048983611075797
For n_clusters = 9 The average silhouette_score using single link is : 0.24590378484479003
For n_clusters = 9 The average silhouette_score using average link is : 0.3158047156300784
For n_clusters = 9 The average silhouette_score using complete link is : 0.2731419390954508
For n_clusters = 9 The average silhouette_score using ward is : 0.30334958060660183
For n_clusters = 10 The average silhouette_score using single link is : 0.28330035015536476
For n_clusters = 10 The average silhouette_score using average link is : 0.33098912903776545
For n_clusters = 10 The average silhouette_score using complete link is : 0.3086401825690758
For n_

Los que funcionan mejor son average link y ward